In [ ]:
import pandas as pd
import numpy as np
import pymssql

In [26]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')

In [54]:
#test connection
query='SELECT Filename, LockPath, LockDomain, LockDate, Username,FullName, Email from [ePDM_Elcam].[dbo].[Documents] INNER JOIN [ePDM_Elcam].[dbo].[Users] on [ePDM_Elcam].[dbo].[Documents].UserID = [ePDM_Elcam].[dbo].[Users].UserID WHERE LockDate IS NOT NULL AND CurrentStatusid IS NOT NULL AND Deleted=0 ORDER BY Username ASC'
#run query
files = pd.read_sql(query, mssql_conn)
files.sort_values(by='LockDate', ascending=False, inplace=True)
files


c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Filename,LockPath,LockDomain,LockDate,Username,FullName,Email
179,VAL-1867_STP BODY 4W 45 FFRDM PN 522511 in mou...,C:\ePDM_Elcam\Validations\Val-1867-STP BODY 4W...,MNENG-ELENA-HPZ,2023-02-16 05:31:22.000,elena.kh,Elena Khaikin,elena.kh@elcam.co.il
10,הזרקה חוזרת לטיפול בנזילות.docx,C:\ePDM_Elcam\Validations\Val-1752-MONOBLOCK 3...,MNENG-ALONA-G7,2023-02-16 05:30:38.080,alona.a,Alona Abramov,alona.a@elcam.co.il
639,PRS-0004-Somatrogon Injector pen-Product Requi...,C:\ePDM_Elcam\System Project\PRS-0004-Somatrog...,ENG-ILAN-HP,2023-02-15 17:02:54.000,ilan.k,Ilan Kachler,ilan.k@elcam.co.il
840,AA24Co78639.SLDPRT,C:\ePDM_Elcam\ManEng\שרטוטי מתקנים\AA - הרכבה ...,MNENG-NEWENGNNE,2023-02-15 16:06:14.000,sergei.h,Sergei Holodovsky,sergei.h@elcam.co.il
832,AA24Co78639.SLDDRW,C:\ePDM_Elcam\ManEng\שרטוטי מתקנים\AA - הרכבה ...,MNENG-NEWENGNNE,2023-02-15 16:05:57.000,sergei.h,Sergei Holodovsky,sergei.h@elcam.co.il
...,...,...,...,...,...,...,...
803,All Parts For manufacturing.xlsx,C:\ePDM_Elcam\ManEng\שרטוטי מתקנים\AA - הרכבה ...,MNENG-NEWENGNNE,2022-01-03 16:16:15.580,sergei.h,Sergei Holodovsky,sergei.h@elcam.co.il
212,בדיקה חשמלית לוט C.pdf,C:\ePDM_Elcam\Validations\Val-1771-new gel cup...,MNENG-EYAL-T165,2022-01-03 14:44:24.170,eyal.sh,eyal.sh,eyal.sh@elcam.co.il
211,בדיקה חשמלית לוט B.pdf,C:\ePDM_Elcam\Validations\Val-1771-new gel cup...,MNENG-EYAL-T165,2022-01-03 14:41:38.607,eyal.sh,eyal.sh,eyal.sh@elcam.co.il
210,בדיקה חשמלית לוט A.pdf,C:\ePDM_Elcam\Validations\Val-1771-new gel cup...,MNENG-EYAL-T165,2022-01-03 14:40:30.240,eyal.sh,eyal.sh,eyal.sh@elcam.co.il


In [61]:

res ={}
for i in range(len(files)):
    row=files.iloc[i]
    if row['Username'] not in res:
        res[row['Username']] ={'email':row['Email'],'fullname':row['FullName'],'count':0}
    res[row['Username']]['count']+=1
#sort by count
res = sorted(res.items(), key=lambda x: x[1]['count'], reverse=True)
username=[]
fullname=[]
email=[]
count_files=[]
for k in res:
    # print(k)
    username.append(k[0])
    fullname.append(k[1]['fullname'])
    email.append(k[1]['email'])
    count_files.append(k[1]['count'])
df = pd.DataFrame({'username':username,'fullname':fullname,'email':email,'count_files':count_files})
df



,username,fullname,email,count_files
0,hen.b,Hen Balik,hen.b@elcam.co.il,180
1,galei,Gal Einhorn,galei@elcam.co.il,145
2,sergei.h,Sergei Holodovsky,sergei.h@elcam.co.il,83
3,ayelet.ts,Ayelet Tal Shahar,ayelet.ts@elcam.co.il,67
4,yair.e,Yair Elbaz,yair.e@elcam.co.il,41
...,...,...,...,...
80,mahir.d,Mahir Dakwar,mahir.d@elcam.co.il,1
81,olga.al,Olga Alexandrova,olga.al@elcam.co.il,1
82,israel.d,Israel Dembus,israel.d@elcam.co.il,1
83,ihsan.z,Ihsan Zaidan,ihsan.z@elcam.co.il,1


In [52]:
df2=pd.read_csv("kimaat.moshlam.csv")
#remove all the rows that have no email address
df2=df2[df2['EmailAddress'].notnull()]
#remove all the rows that the email address is not end with @elcam.co.il
df2=df2[df2['EmailAddress'].str.endswith('@elcam.co.il')]
#convert all the email address to lower case
df2['EmailAddress']=df2['EmailAddress'].str.lower()



0